# Solving 3-SAT Problem with Hybrid Quantum-Classical solver 

**Author:** Xinghui Jia \& Siwei Tan

**Date:** 9/4/2024

Based on paper "[Janus-CT: A Hybrid Approach for 3-SAT Problems by Integrating Quantum Annealer with CDCL][1]"

[1]: https://ieeexplore.ieee.org/abstract/document/10071022/


A propositional satisfiability (SAT) problem is to find an assignment for each variable to satisfy a given Boolean formula. A typical SAT problem is composed of multiple clauses represented as a disjunction of Boolean variables. 3-SAT problem is a special case of the SAT problem that has no more than three variables in each clause. 

3-SAT problem is a fundamental problem in various applications, such as artificial intelligence, circuit analysis, and protein structure prediction. Since the 3-SAT problem is an NP-complete problem, the time complexity of classical algorithms increases exponentially with the number of clauses and variables. We observe that the classical CDCL algorithm has the advantage of solving larger problems, while QA is more suitable for solving small but hard problems. To combine their advantages, we introduce Janus-SAT in this notebook, a hybrid approach that integrates quantum annealing (QA) with the classical Conflict-Driven Clause Learning (CDCL) algorithm to enable end-to-end acceleration for solving 3-SAT problems. 


<div style="text-align:center;">
    <img src="./pictures/4-1.hyqsat.png"  width="45%" height="45%">
</div>

The figure above shows the workflow of Janus-CT. It features a cross-iterative process. The CDCL algorithm searches for the solution. During the search, it identifies and sends the hard sub-problem to QA. Janus-CT designed a fast compilation flow to embed the problem and optimize the noise. It also designs multiple strategies to guide the search of CDCL using the QA results.

In [1]:
import sys
import os
import logging

sys.path.append('..')

from janusq.hyqsat import solve_by_janusct, solve_by_minisat

In [2]:
# Configure the model
file_path = "cnf_examples/UF100/uf100-01.cnf"  # input cnf flie
cpu_lim = 0  # limit the cpu time (s). 0 means infinite
mem_lim = 0  # limit the memory 0 means infinite
strictp = True  # isStrict

### Using Janus-SAT API to Solve the Problem

We can use a QA simulator or real-world QA hardware to solve the SAT roblem. For example, here we use the simualtor.

In [3]:
# solve by Janus-SAT
solve_by_janusct(file_path, verb=False, cpu_lim=cpu_lim, mem_lim=mem_lim)

{'restarts': 1,
 'conflicts': 7,
 'conflict cost': 0.077,
 'decisions': 1,
 'decisions cost': 0.034,
 'propagations': 409,
 'propagations cost': 0.383,
 'conflict literals': 61,
 'actual CPU time': 7322.97,
 'solving time': 4.38,
 'annealing time': 3.72,
 'simulation time': 1248.29,
 'quantum count': 31,
 'quantum success number': 7,
 'quantum conflict number': 24,
 'quantum one time solve number': 0,
 'is satisfiable': True,
 'is sat': True}

Note that when using the simulator, the solving time is estimated as (CDCL time + number of QA $\times$ 120 $\mu s$).

### Comparision to MiniSAT

We compare it to the MiniSAT solver.

In [4]:
# solve by minisat
solve_by_minisat(file_path, verb=False, cpu_lim=cpu_lim, mem_lim=mem_lim)

{'restarts': 3,
 'conflicts': 384,
 'conflict cost': 2.549,
 'decisions': 463,
 'decisions cost': 0.427,
 'propagations': 9130,
 'propagations cost': 5.595,
 'conflict literals': 2416,
 'actual CPU time': 101.954,
 'solving time': 10.216,
 'annealing time': 0.0,
 'simulation time': 0.0,
 'quantum count': 0,
 'quantum success number': 0,
 'quantum conflict number': 0,
 'quantum one time solve number': 0,
 'is satisfiable': True,
 'is sat': True}

In [5]:
all_result = {}
dir_name = 'cnf_examples/test'
result_key = ['restarts', 'conflicts', 'conflict cost', 'decisions', 'decisions cost', 'propagations', 'propagations cost', 'conflict literals', 'actual CPU time', 'solving time']
for filename in os.listdir(dir_name):
    janusq_res = solve_by_janusct(os.path.join(dir_name, filename), verb=False)
    minisat_res = solve_by_minisat(os.path.join(dir_name, filename), verb=False)
    for key in result_key:
        if key not in all_result:
            all_result[key] = {
                'janus': 0,
                'minisat': 0
            }
        else:
            all_result[key]['janus'] += janusq_res[key]
            all_result[key]['minisat'] += minisat_res[key]

print(all_result)    

2024-04-22 07:37:25,346 - INFO: WARNING: for repeatability, setting FPU to use double precision
2024-04-22 07:37:25,347 - INFO: sys.path.append('/home/jiaxinghui/JanusQ/JanusQ/examples/../janusq/hyqsat/python/')
2024-04-22 07:37:25,347 - INFO: Quantum method:
2024-04-22 07:37:25,348 - INFO: /home/jiaxinghui/JanusQ/JanusQ/examples/cnf_examples/test/uf100-03.cnf
2024-04-22 07:37:26,418 - INFO: ============================[ Problem Statistics ]=============================
2024-04-22 07:37:26,419 - INFO: |                                                                             |
2024-04-22 07:37:26,420 - INFO: |  Number of variables:           100                                         |
2024-04-22 07:37:26,421 - INFO: |  Number of clauses:             430                                         |
2024-04-22 07:37:26,421 - INFO: ============================[ Search Statistics ]==============================
2024-04-22 07:37:26,421 - INFO: | Conflicts |          ORIGINAL         |    

{'restarts': {'janus': 5, 'minisat': 9}, 'conflicts': {'janus': 49, 'minisat': 755}, 'conflict cost': {'janus': 0.371, 'minisat': 4.816}, 'decisions': {'janus': 19, 'minisat': 958}, 'decisions cost': {'janus': 0.121, 'minisat': 0.895}, 'propagations': {'janus': 1459, 'minisat': 17399}, 'propagations cost': {'janus': 1.27, 'minisat': 10.592}, 'conflict literals': {'janus': 293, 'minisat': 4558}, 'actual CPU time': {'janus': 35936.05, 'minisat': 331.65999999999997}, 'solving time': {'janus': 19.488, 'minisat': 19.399}}


In [6]:
import pandas as pd

df = pd.DataFrame(all_result)
df

,restarts,conflicts,conflict cost,decisions,decisions cost,propagations,propagations cost,conflict literals,actual CPU time,solving time
janus,5,49,0.371,19,0.121,1459,1.270,293,35936.05,19.488
minisat,9,755,4.816,958,0.895,17399,10.592,4558,331.66,19.399
